# Phoenix DB Client Example

### You can load a dataset into a Phoenix table via the PhoenixDB library  

#### First, create a database connection. Notice Phoenix credentials are saved into your CML Project as environment variables. 
#### To add yours, navigate to Project Settings -> Advanced -> Add the variables for your user and restart the CML session.

Helper Functions

In [101]:
import pandas as pd

In [128]:
class Db:
    def __init__(self):
        opts = {}
        opts['authentication'] = 'BASIC'
        opts['avatica_user'] = os.environ["WORKLOAD_USER"]
        opts['avatica_password'] = os.environ["WORKLOAD_PASSWORD"]
        database_url = os.environ["OPDB_ENDPOINT_PSE"]
        self.TABLENAME = "test_table_paul"
        self.conn = phoenixdb.connect(database_url, autocommit=True,**opts)
        self.curs = self.conn.cursor()
        
    def create_linkage_table(self):
        
        query = """
        CREATE TABLE IF NOT EXISTS "CML_WORKSHOP_TABLE" (
        unique_id VARCHAR,
        first_name VARCHAR,
        surname VARCHAR,
        dob VARCHAR,
        city  VARCHAR,
        email VARCHAR,
        groupp VARCHAR CONSTRAINT my_pk PRIMARY KEY (unique_id))
        """
        
        self.curs.execute(query)
        
    def upsert_linkage(self, data):

        sql = """upsert into "CML_WORKSHOP_TABLE" \
             (unique_id ,first_name,surname,dob,city,email,groupp) \
             values (?,?,?,?,?,?,?)"""
        #print(data)
        self.curs.executemany(sql,data)
        self.conn.commit()
        
    def get_data(self):

        query = f"SELECT count(*) FROM CML_WORKSHOP_TABLE"

        model.curs.execute(query)
        rows = model.curs.fetchall()

        return rows

In [116]:
def upsert_data(data, records=100):
        total_records=0
        header = True
        rows = []
        i=1
        model=Db()
        for index, row in data.iterrows():

            rows.append ([f"{row['unique_id']}",\
                      f"{row['first_name']}",f"{row['surname']}",\
                      f"{row['dob']}",f"{row['city']}", \
                      f"{row['email']}", f"{row['group']}"])
            total_records=total_records+1

            if i < records + 1 :   
                i=i+1
            else :
                model.upsert_linkage(rows)
                rows = []
                i=1
                print (f"Ingested {total_records} records")

        if len(rows) > 0 :
            model.upsert_linkage(rows)

        print (f"Ingested {total_records} records")

#### In order to demonstrate this, we will load the sample dataset into a Phoenix table.

In [106]:
df= pd.read_parquet("data/fake_df_l.parquet", engine="pyarrow")

In [107]:
df.head()

,unique_id,first_name,surname,dob,city,email,group
0,0,Julia,None,2015-10-29,London,hannah88@powers.com,0
1,4,oNah,Watson,2008-03-23,Bolton,matthew78@ballard-mcdonald.net,1
2,13,Molly,Bell,2002-01-05,Peterborough,None,2
3,15,Alexander,Amelia,1983-05-19,Glasgow,ic-mpbell@allealewis.org,3
4,20,Ol vri,ynnollC,1972-03-08,Plymouth,derekwilliams@norris.com,4


In [129]:
import logging
logging.basicConfig( level=logging.DEBUG)

import os
import phoenixdb
model=Db()

DEBUG:phoenixdb.avatica.client:Sending request
connection_id: "372bef52-f42b-44c6-b536-c2b6015150e1"

DEBUG:phoenixdb.avatica.client:POST https://cod-ojgyjyaq708e-gateway0.pse-722.ylcu-atmi.cloudera.site/cod-ojgyjyaq708e/cdp-proxy-api/avatica b'\n?org.apache.calcite.avatica.proto.Requests$OpenConnectionRequest\x12&\n$372bef52-f42b-44c6-b536-c2b6015150e1' {'content-type': 'application/x-google-protobuf'}
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): cod-ojgyjyaq708e-gateway0.pse-722.ylcu-atmi.cloudera.site:443
DEBUG:urllib3.connectionpool:https://cod-ojgyjyaq708e-gateway0.pse-722.ylcu-atmi.cloudera.site:443 "POST /cod-ojgyjyaq708e/cdp-proxy-api/avatica HTTP/1.1" 200 134
DEBUG:phoenixdb.avatica.client:Received response
name: "org.apache.calcite.avatica.proto.Responses$OpenConnectionResponse"
wrapped_message: "\n?\n=cod-ojgyjyaq708e-worker4.pse-722.ylcu-atmi.cloudera.site:8765"

DEBUG:phoenixdb.avatica.client:Sending request
connection_id: "372bef52-f42b-44c6-b536-c2b601

In [113]:
import logging
logging.basicConfig( level=logging.DEBUG)

#model.drop_stocks_table()
model.create_linkage_table()

DEBUG:phoenixdb.avatica.client:Sending request
connection_id: "b68dcc2e-216b-4f02-ac9e-3c5836efaa6a"

DEBUG:phoenixdb.avatica.client:POST https://cod-ojgyjyaq708e-gateway0.pse-722.ylcu-atmi.cloudera.site/cod-ojgyjyaq708e/cdp-proxy-api/avatica b'\n@org.apache.calcite.avatica.proto.Requests$CreateStatementRequest\x12&\n$b68dcc2e-216b-4f02-ac9e-3c5836efaa6a' {'content-type': 'application/x-google-protobuf'}
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): cod-ojgyjyaq708e-gateway0.pse-722.ylcu-atmi.cloudera.site:443
DEBUG:urllib3.connectionpool:https://cod-ojgyjyaq708e-gateway0.pse-722.ylcu-atmi.cloudera.site:443 "POST /cod-ojgyjyaq708e/cdp-proxy-api/avatica HTTP/1.1" 200 175
DEBUG:phoenixdb.avatica.client:Received response
name: "org.apache.calcite.avatica.proto.Responses$CreateStatementResponse"
wrapped_message: "\n$b68dcc2e-216b-4f02-ac9e-3c5836efaa6a\020\014\032?\n=cod-ojgyjyaq708e-worker1.pse-722.ylcu-atmi.cloudera.site:8765"

DEBUG:phoenixdb.avatica.client:Sending req

#### Next, navigate to the COD Experience in CDP and open the Hue browser. 
#### Check that the table is there.

In [117]:
import logging
logging.basicConfig( level=logging.DEBUG)

upsert_data(df)

DEBUG:phoenixdb.avatica.client:Sending request
connection_id: "cdf377b2-b908-49b1-8b12-85a9b13f5272"

DEBUG:phoenixdb.avatica.client:POST https://cod-ojgyjyaq708e-gateway0.pse-722.ylcu-atmi.cloudera.site/cod-ojgyjyaq708e/cdp-proxy-api/avatica b'\n?org.apache.calcite.avatica.proto.Requests$OpenConnectionRequest\x12&\n$cdf377b2-b908-49b1-8b12-85a9b13f5272' {'content-type': 'application/x-google-protobuf'}
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): cod-ojgyjyaq708e-gateway0.pse-722.ylcu-atmi.cloudera.site:443
DEBUG:urllib3.connectionpool:https://cod-ojgyjyaq708e-gateway0.pse-722.ylcu-atmi.cloudera.site:443 "POST /cod-ojgyjyaq708e/cdp-proxy-api/avatica HTTP/1.1" 200 134
DEBUG:phoenixdb.avatica.client:Received response
name: "org.apache.calcite.avatica.proto.Responses$OpenConnectionResponse"
wrapped_message: "\n?\n=cod-ojgyjyaq708e-worker4.pse-722.ylcu-atmi.cloudera.site:8765"

DEBUG:phoenixdb.avatica.client:Sending request
connection_id: "cdf377b2-b908-49b1-8b12-85a9b1

DEBUG:phoenixdb.avatica.client:POST https://cod-ojgyjyaq708e-gateway0.pse-722.ylcu-atmi.cloudera.site/cod-ojgyjyaq708e/cdp-proxy-api/avatica b'\n=org.apache.calcite.avatica.proto.Requests$ExecuteBatchRequest\x12\xd6N\n$cdf377b2-b908-49b1-8b12-85a9b13f5272\x1aY\n\x05\x08\x15\x1a\x010\n\n\x08\x15\x1a\x06Julia \n\x08\x08\x15\x1a\x04None\n\x0e\x08\x15\x1a\n2015-10-29\n\n\x08\x15\x1a\x06London\n\x17\x08\x15\x1a\x13hannah88@powers.com\n\x05\x08\x15\x1a\x010\x1ad\n\x05\x08\x15\x1a\x014\n\x08\x08\x15\x1a\x04oNah\n\n\x08\x15\x1a\x06Watson\n\x0e\x08\x15\x1a\n2008-03-23\n\n\x08\x15\x1a\x06Bolton\n"\x08\x15\x1a\x1ematthew78@ballard-mcdonald.net\n\x05\x08\x15\x1a\x011\x1aQ\n\x06\x08\x15\x1a\x0213\n\n\x08\x15\x1a\x06Molly \n\x08\x08\x15\x1a\x04Bell\n\x0e\x08\x15\x1a\n2002-01-05\n\x10\x08\x15\x1a\x0cPeterborough\n\x08\x08\x15\x1a\x04None\n\x05\x08\x15\x1a\x012\x1af\n\x06\x08\x15\x1a\x0215\n\r\x08\x15\x1a\tAlexander\n\x0b\x08\x15\x1a\x07Amelia \n\x0e\x08\x15\x1a\n1983-05-19\n\x0b\x08\x15\x1a\x07Glasgo

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): cod-ojgyjyaq708e-gateway0.pse-722.ylcu-atmi.cloudera.site:443
DEBUG:urllib3.connectionpool:https://cod-ojgyjyaq708e-gateway0.pse-722.ylcu-atmi.cloudera.site:443 "POST /cod-ojgyjyaq708e/cdp-proxy-api/avatica HTTP/1.1" 200 274
DEBUG:phoenixdb.avatica.client:Received response
name: "org.apache.calcite.avatica.proto.Responses$ExecuteBatchResponse"
wrapped_message: "\n$cdf377b2-b908-49b1-8b12-85a9b13f5272\032e\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001*?\n=cod-ojgyjyaq708e-worker4.pse-722.ylcu-atmi.cloudera.site:8765"

DEBUG:phoenixdb.avatica.client:Sending request
connecti

Ingested 101 records


DEBUG:urllib3.connectionpool:https://cod-ojgyjyaq708e-gateway0.pse-722.ylcu-atmi.cloudera.site:443 "POST /cod-ojgyjyaq708e/cdp-proxy-api/avatica HTTP/1.1" 200 134
DEBUG:phoenixdb.avatica.client:Received response
name: "org.apache.calcite.avatica.proto.Responses$CloseStatementResponse"
wrapped_message: "\n?\n=cod-ojgyjyaq708e-worker4.pse-722.ylcu-atmi.cloudera.site:8765"

DEBUG:phoenixdb.avatica.client:Sending request
connection_id: "cdf377b2-b908-49b1-8b12-85a9b13f5272"
statement_id: 1
updates {
  parameter_values {
    type: STRING
    string_value: "603"
  }
  parameter_values {
    type: STRING
    string_value: "None"
  }
  parameter_values {
    type: STRING
    string_value: "Ky"
  }
  parameter_values {
    type: STRING
    string_value: "1996-05-13"
  }
  parameter_values {
    type: STRING
    string_value: "oeteborPugh"
  }
  parameter_values {
    type: STRING
    string_value: "seancollins@dean.com"
  }
  parameter_values {
    type: STRING
    string_value: "101"
  }
}
upd

DEBUG:phoenixdb.avatica.client:POST https://cod-ojgyjyaq708e-gateway0.pse-722.ylcu-atmi.cloudera.site/cod-ojgyjyaq708e/cdp-proxy-api/avatica b'\n=org.apache.calcite.avatica.proto.Requests$ExecuteBatchRequest\x12\x93?\n$cdf377b2-b908-49b1-8b12-85a9b13f5272\x10\x01\x1a_\n\x07\x08\x15\x1a\x03603\n\x08\x08\x15\x1a\x04None\n\x06\x08\x15\x1a\x02Ky\n\x0e\x08\x15\x1a\n1996-05-13\n\x0f\x08\x15\x1a\x0boeteborPugh\n\x18\x08\x15\x1a\x14seancollins@dean.com\n\x07\x08\x15\x1a\x03101\x1a^\n\x07\x08\x15\x1a\x03612\n\n\x08\x15\x1a\x06Caleb \n\x07\x08\x15\x1a\x03Kan\n\x0e\x08\x15\x1a\n2000-10-09\n\r\x08\x15\x1a\tWirangton\n\x16\x08\x15\x1a\x12vmoreno@ussell.zib\n\x07\x08\x15\x1a\x03102\x1aj\n\x07\x08\x15\x1a\x03614\n\n\x08\x15\x1a\x06Wright\n\t\x08\x15\x1a\x05Jude \n\x0e\x08\x15\x1a\n2017-03-28\n\x0b\x08\x15\x1a\x07Belfast\n"\x08\x15\x1a\x1elynnchapman@crawfard-lozon.com\n\x07\x08\x15\x1a\x03103\x1a_\n\x07\x08\x15\x1a\x03615\n\x0c\x08\x15\x1a\x08Jessica \n\x08\x08\x15\x1a\x04Carr\n\x0e\x08\x15\x1a\n1972

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): cod-ojgyjyaq708e-gateway0.pse-722.ylcu-atmi.cloudera.site:443
DEBUG:urllib3.connectionpool:https://cod-ojgyjyaq708e-gateway0.pse-722.ylcu-atmi.cloudera.site:443 "POST /cod-ojgyjyaq708e/cdp-proxy-api/avatica HTTP/1.1" 200 255
DEBUG:phoenixdb.avatica.client:Received response
name: "org.apache.calcite.avatica.proto.Responses$ExecuteBatchResponse"
wrapped_message: "\n$cdf377b2-b908-49b1-8b12-85a9b13f5272\020\001\032P\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001*?\n=cod-ojgyjyaq708e-worker4.pse-722.ylcu-atmi.cloudera.site:8765"

DEBUG:phoenixdb.avatica.client:Sending request
connection_id: "cdf377b2-b908-49b1-8b12-85a9b13f5272"

DEBUG:phoenixdb.avatica.clien

Ingested 181 records


DEBUG:urllib3.connectionpool:https://cod-ojgyjyaq708e-gateway0.pse-722.ylcu-atmi.cloudera.site:443 "POST /cod-ojgyjyaq708e/cdp-proxy-api/avatica HTTP/1.1" 200 135
DEBUG:phoenixdb.avatica.client:Received response
name: "org.apache.calcite.avatica.proto.Responses$CloseConnectionResponse"
wrapped_message: "\n?\n=cod-ojgyjyaq708e-worker4.pse-722.ylcu-atmi.cloudera.site:8765"



#### Finally, check that the upsert worked:

In [130]:
model.get_data()

DEBUG:phoenixdb.avatica.client:Sending request
connection_id: "372bef52-f42b-44c6-b536-c2b6015150e1"

DEBUG:phoenixdb.avatica.client:POST https://cod-ojgyjyaq708e-gateway0.pse-722.ylcu-atmi.cloudera.site/cod-ojgyjyaq708e/cdp-proxy-api/avatica b'\n@org.apache.calcite.avatica.proto.Requests$CreateStatementRequest\x12&\n$372bef52-f42b-44c6-b536-c2b6015150e1' {'content-type': 'application/x-google-protobuf'}
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): cod-ojgyjyaq708e-gateway0.pse-722.ylcu-atmi.cloudera.site:443
DEBUG:urllib3.connectionpool:https://cod-ojgyjyaq708e-gateway0.pse-722.ylcu-atmi.cloudera.site:443 "POST /cod-ojgyjyaq708e/cdp-proxy-api/avatica HTTP/1.1" 200 175
DEBUG:phoenixdb.avatica.client:Received response
name: "org.apache.calcite.avatica.proto.Responses$CreateStatementResponse"
wrapped_message: "\n$372bef52-f42b-44c6-b536-c2b6015150e1\020\002\032?\n=cod-ojgyjyaq708e-worker4.pse-722.ylcu-atmi.cloudera.site:8765"

DEBUG:phoenixdb.avatica.client:Sending req

[[181]]